In [3]:
# Download Amazon 2018 toy & games review and metadata
%cd ../Data/
%run download_amazon_data.py
%cd ../alex_fuzzy_matching/

/home/alex/Documents/erdos/summer-2025-safeify/Data
File already exists.
File already exists.
/home/alex/Documents/erdos/summer-2025-safeify/alex_fuzzy_matching


In [4]:
import pandas as pd

In [5]:
# Import first 100,000 entries of amazon metadata into a pandas dataframe.
# Obviously we'd want to do this on the full dataset, but my computer struggled importing the full dataset due to RAM limitations
meta_chunks=pd.read_json("../Data/amazon_meta.json",lines=True,chunksize=100_000)
metadata=next(meta_chunks)

In [6]:
# Import CPSC IncidentReports
reports=pd.read_csv("../Data/IncidentReports.csv",encoding='cp1252')

# Filter to reports in the Toys and Children category. (There's also a baby category.)
reports=reports.loc[reports['Product Category']=='Toys & Children']

# Filter to reports of items manufactured before October 2018,
# since items manufuctered after this data won't appear in the amazon dataset
reports['Date Manufactured']=pd.to_datetime(reports['Date Manufactured'])
report=reports[reports['Date Manufactured']<='Oct 2018']

/tmp/ipykernel_62849/612986380.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  reports=pd.read_csv("../Data/IncidentReports.csv",encoding='cp1252')


In [7]:
# Features we can use for fuzzy matching.

print(metadata.columns)
print(reports.columns)

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')
Index(['Report No.', 'Report Date',
       'Sent to Manufacturer / Importer / Private Labeler', 'Publication Date',
       'Category of Submitter', 'Product Description', 'Product Category',
       'Product Sub Category', 'Product Type', 'Product Code',
       'Manufacturer / Importer / Private Labeler Name', 'Brand',
       'Model Name or Number', 'Serial Number', 'UPC', 'Date Manufactured',
       'Manufacturer Date Code', 'Retailer', 'Retailer State', 'Purchase Date',
       'Purchase Date Is Estimate', 'Incident Description', 'City', 'State',
       'ZIP', 'Location', '(Primary) Victim Severity',
       '(Primary) Victim's Sex', 'My Relation To The (Primary) Victim',
       '(Primary) Victim's Age (years)', 'Submitter Has Product',
 

In [8]:
import rapidfuzz as rf


In [9]:

def best_match(query,choices):
    """Function returning best match of "query" in "choices"
    Returns:
        tuple: A tuple consisting of the following elemnts:
            - The best match choice in choices
            - A normalized "edit distance"
            - The index of the best match.
    """
    return rf.process.extractOne(query,choices,scorer=rf.fuzz.ratio)


In [52]:
reports

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,...,Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
5,20250225-EC92A-5152930,2/25/2025,5/5/2025,5/19/2025,Consumer,sleeping aid,Toys & Children,Toys,"Dolls, Plush Toys, and Action Figures (1394)",1394,...,Yes,No,NaN,NaN,No,Yes,NaN,Sent email,NaN,NaN
18,20250430-BD4AC-2147331626,4/30/2025,5/2/2025,5/16/2025,Consumer,100 Pcs Large Stepped Pyramid Wooden Stacking ...,Toys & Children,Toys,"Blocks, Stacking Toys or Pull Toys (1326)",1326,...,No,Yes,NaN,NaN,NaN,No,NaN,Blocks chipped when played with. Toy was throw...,NaN,NaN
29,20250426-11C99-2147331693,4/26/2025,5/1/2025,5/15/2025,Consumer,Magnetic building tiles by Anbalulu\r\nMagneti...,Toys & Children,Toys,Building Sets (1345),1345,...,Yes,No,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
35,20250425-00047-2147331727,4/25/2025,5/1/2025,5/15/2025,Consumer,Easter Battery Operated Bubble Wand (way to ce...,Toys & Children,Toys,"Toys, Not Elsewhere Classified (1381)",1381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,20250419-2B92E-2147331854,4/19/2025,4/24/2025,5/15/2025,Consumer,PetsBoro Bunnypal,Toys & Children,Toys,"Dolls, Plush Toys, and Action Figures (1394)",1394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61891,20110312-F8C6F-2147481550,3/12/2011,NaN,4/6/2011,Consumer,Child's toy stroller,Toys & Children,Toys,"Dolls, Plush Toys, and Action Figures (1394)",1394,...,Yes,No,NaN,NaN,No,No,No,NaN,NaN,NaN
61908,20110312-200A5-2147481542,3/12/2011,NaN,4/2/2011,Consumer,Recalled Graco Stroller (the one with fingerti...,Toys & Children,Strollers & Car Seats,Baby Strollers (1522),1522,...,No,No,NaN,NaN,No,Yes,NaN,Reported the incident to Graco.,(3/25/2011) Newell Rubbermaid Inc.: Graco Chil...,NaN
61939,20110311-C0123-2147481592,3/11/2011,NaN,4/1/2011,Consumer,Friendly Faces Royal Princess Rattle,Toys & Children,Toys,Baby Rattles (1517),1517,...,No,No,NaN,NaN,No,Yes,NaN,"I contacted them, they requested that I send t...",NaN,NaN
61943,20110311-E518D-1170349,3/11/2011,NaN,4/1/2011,Consumer,Airzone Pop stick. Bright/ lime green with bla...,Toys & Children,Toys,Pogo Sticks (1310),1310,...,Yes,No,NaN,No,No,No,No,NaN,(3/31/2011) BRAVO SPORTS: Thank you for transm...,NaN


In [47]:
# Restricting to N sample reports to try and match them
N=100_000
reports_sample=reports.iloc[:N].copy()

In [49]:
reports_sample['best_brand_match_data']=reports_sample['Brand'].apply(lambda x: best_match(x,metadata['brand']))


In [ ]:
reports_sample['Best Brand Match']=reports_sample['best_brand_match_data'].apply(lambda x : x[0] if x!=None else None)
reports_sample['Brand Match Score']=reports_sample['best_brand_match_data'].apply(lambda x : x[1] if x!=None else None)
reports_sample[['Brand','Best Brand Match','Brand Match Score']][reports_sample['Brand Match Score']>90]

,Brand,Best Brand Match,Brand Match Score
500,Fisher Price,Fisher Price,100.000000
518,Sesame street,Sesame Street,92.307692
592,Sassy,Sassy,100.000000
646,Kid Connection,Kid Connection,100.000000
671,Fisher Price,Fisher Price,100.000000
...,...,...,...
61880,Baby Trend,Baby Trend,95.238095
61891,Circo,Circo,100.000000
61908,Graco,Graco,100.000000
61939,Yookidoo,Yookidoo,100.000000


In [51]:
reports_sample['best_title_match_data']=reports_sample['Product Description'].apply(lambda x: rf.process.extractOne(x,metadata['title'],scorer=rf.fuzz.ratio))


In [ ]:
reports_sample['Best Desc. Match']=reports_sample['best_title_match_data'].apply(lambda x : x[0] if x!=None else None)
reports_sample['Desc. Match Score']=reports_sample['best_title_match_data'].apply(lambda x : x[1] if x!=None else None)
reports_sample[['Product Description','Best Desc. Match','Desc. Match Score']][reports_sample['Desc. Match Score']>80]


,Product Description,Best Desc. Match,Desc. Match Score
74,Bubble blaster,Bubble Beater,81.481481
6398,Melissa & Doug Take-Along Tool Kit Wooden Cons...,Melissa &amp; Doug Take-Along Tool Kit Wooden ...,84.722222
7248,Daron Spirit Airlines Airport Play Set,Daron Alaska Airlines Airport Play Set,84.210526
23368,Hula Hair Barbie,1996 Hula Hair Barbie,86.486486
24700,Snoopy SnoCone Machine,Snoopy Sno Cone Machine,97.777778
24986,Umbrella stroller,Umbrella Doll Stroller,82.051282
27585,Playhut: crawl n play,Playhut Crawl N Play,82.926829
32795,Kidoozie Little Doctor Kit - Includes Stethosc...,Kidoozie Little Doctor Kit &ndash; Includes St...,95.238095
37442,Vtech Discovery Nursery Farm,VTech - Discovery Nursery Farm,93.103448
43995,Crystal mining kit,4M Crystal Mining Kit,82.051282


In [63]:
matches=reports_sample[['Product Description','Best Desc. Match','Brand','Best Brand Match','Desc. Match Score','Brand Match Score']][(reports_sample['Desc. Match Score']>70 )& (reports_sample['Brand Match Score']>70)]
matches.to_csv("potential_matches.csv")